In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle("../data/preprocess_train_dimbat_2011.pkl")
df.sample(5)

,id,text,relevance,text_clean,lemmas,text_lemma
729,None,thank you all for your responses to my countle...,1,thank you all for your responses to my countle...,"[thank, response, countless, tweet, twitpic, j...",thank response countless tweet devastation har...
2636,60350187654217728,<USER> an apartment? hope there's room for the...,0,an apartment? hope there's room for the bike ,"[apartment, hope, room, bike]",apartment hope room bike
875,None,severe thunderstorm  <REPEAT> tornado warning...,1,severe thunderstorm  tornado warning  the ...,"[severe, thunderstorm, tornado, warn, weather,...",severe thunderstorm tornado warn weather roll ...
2254,459433811806154752,<USER> @southwestair be glad it's not @british...,0,@southwestair be glad it's not @britishairways,[glad],glad
1589,None,yea now we're under a tornado warning. this is...,1,yea now we're under a tornado warning. this is...,"[yea, tornado, warn, exhaust]",yea tornado warn exhaust


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3752 entries, 0 to 3751
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1876 non-null   object
 1   text        3752 non-null   object
 2   relevance   3752 non-null   int64 
 3   text_clean  3752 non-null   object
 4   lemmas      3752 non-null   object
 5   text_lemma  3752 non-null   object
dtypes: int64(1), object(5)
memory usage: 176.0+ KB


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline
from sklearn.metrics import confusion_matrix, classification_report

In [5]:
tweets = list(df["text_clean"])

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
#tf_model = TFAutoModelForSequenceClassification.from_pretrained("../models/dimbat_disaster_distilbert_model")

In [119]:
tokenizer = AutoTokenizer.from_pretrained("sacculifer/dimbat_disaster_distilbert")

tf_model = TFAutoModelForSequenceClassification.from_pretrained("sacculifer/dimbat_disaster_distilbert")

Some layers from the model checkpoint at sacculifer/dimbat_disaster_distilbert were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at sacculifer/dimbat_disaster_distilbert and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
classifier = pipeline("text-classification", tokenizer=tokenizer, model=tf_model)

In [8]:
output = classifier(tweets)

In [9]:
y_pred = pd.get_dummies(d["label"] for d in output)["LABEL_1"]

In [10]:
print(confusion_matrix(df["relevance"], y_pred))
print(classification_report(df["relevance"], y_pred))

[[1804   72]
 [   4 1872]]
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1876
           1       0.96      1.00      0.98      1876

    accuracy                           0.98      3752
   macro avg       0.98      0.98      0.98      3752
weighted avg       0.98      0.98      0.98      3752



In [11]:
df["predict"] = y_pred

In [12]:
df.sample(10)

,id,text,relevance,text_clean,lemmas,text_lemma,predict
2821,960637525483192320,<USER> reb is a cmb anti too,0,reb is a cmb anti too,"[reb, cmb, anti]",reb cmb anti,0
1510,None,<USER> pretty good lady. we are under a tornad...,1,pretty good lady. we are under a tornado watc...,"[pretty, good, lady, tornado, watch, finger, c...",pretty good lady tornado watch finger cross,1
2466,748364654783664128,i swear 😂 <URL>,0,i swear 😂,"[I, swear]",I swear,0
479,None,"jeff piotrowski has been streaming live ""chase...",1,"jeff piotrowski has been streaming live ""chase...","[jeff, piotrowski, stream, live, chasercam, to...",stream live near intense time,1
3583,771893014516072448,<USER> <URL>,0,,[],,0
3184,436537863296266240,get da hottest mag out now <USER> issue <HASHT...,0,get da hottest mag out now issue with cli...,"[da, hot, mag, issue, click]",da hot mag issue click,0
2359,556277494987644928,that was a dick move,0,that was a dick move,[dick],dick,0
21,None,tornado warning for tarrant county. storm spot...,1,tornado warning for tarrant county. storm spot...,"[tornado, warn, tarrant, county, storm, spotte...",tornado warn storm spotter report funnel cloud...,1
3252,60349981055389696,another tuesday night <USER> another few hours...,0,another tuesday night another few hours shave...,"[tuesday, night, hour, shave, end, life, I, pr...",shave end life I probably enjoy,0
2292,53599003803254784,you cant rap or sing  <REPEAT> but yet you al...,0,you cant rap or sing  but yet you always cre...,"[rap, sing, create, song, goo, ou]",rap sing create song goo ou,0


In [ ]:
#df.to_pickle("../data/preprocess_train_dimbat_2018_pred.pkl")

In [ ]:
#df_new = pd.read_pickle("../data/preprocess_train_dimbat_2018_pred.pkl")

In [ ]:
#df_new.sample(10)

In [104]:
text = "CDC's concern over England London, Germany Berlin, and Shanghai Monkeypox outbreak and more"

In [54]:
output_test = classifier(text)

In [55]:
output_test

[{'label': 'LABEL_1', 'score': 0.9833018183708191}]

In [56]:
output_test[0]['label']

'LABEL_1'

In [10]:
import pycountry

In [11]:
from geopy.geocoders import Nominatim

In [12]:
tokenizer_geo = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [114]:
from transformers import AutoModelForTokenClassification, AutoModelForSequenceClassification

In [15]:
model_geo = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [17]:
nlp = pipeline("ner", model=model_geo, tokenizer=tokenizer_geo, aggregation_strategy='average')

In [105]:
ner_results = nlp(text)

In [106]:
ner_results

[{'entity_group': 'ORG',
  'score': 0.49982148,
  'word': 'CDC',
  'start': 0,
  'end': 3},
 {'entity_group': 'LOC',
  'score': 0.7980037,
  'word': 'England London',
  'start': 19,
  'end': 33},
 {'entity_group': 'LOC',
  'score': 0.8422159,
  'word': 'Germany Berlin',
  'start': 35,
  'end': 49},
 {'entity_group': 'LOC',
  'score': 0.99629694,
  'word': 'Shanghai',
  'start': 55,
  'end': 63}]

In [107]:
ner_results = pd.DataFrame(ner_results)

In [108]:
ner_results

,entity_group,score,word,start,end
0,ORG,0.499821,CDC,0,3
1,LOC,0.798004,England London,19,33
2,LOC,0.842216,Germany Berlin,35,49
3,LOC,0.996297,Shanghai,55,63


In [109]:
ner_loc = ner_results[ner_results["entity_group"] == "LOC"]

In [110]:
ner_loc

,entity_group,score,word,start,end
1,LOC,0.798004,England London,19,33
2,LOC,0.842216,Germany Berlin,35,49
3,LOC,0.996297,Shanghai,55,63


In [112]:
if not ner_loc.empty :
  i = 0
  for x in ner_loc["score"]:
    if x > 0.5:
       print(ner_loc["word"].iloc[i])
    i += 1

England London
Germany Berlin
Shanghai


In [115]:
tokenizer2 = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
demotion_model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

In [116]:
classifier3 = pipeline("text-classification", tokenizer=tokenizer2, model=demotion_model)

In [117]:
emo_results = classifier3(text)

In [118]:
emo_results

[{'label': 'fear', 'score': 0.9551898241043091}]

In [121]:
import colorama

In [122]:
from colorama import Fore